In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os, random
import numpy as np

from torchvision import transforms, datasets


"""
[주의!]
상단 코드는 수정하지 마세요
"""
###################################################################


class MyModel(nn.Module):
############# 해당 부분을 구현하시면 됩니다#########################
    def __init__(self, input_dim, hidden_dim):
        super().__init__()        
        
    def forward(self, x):
        return x


###################### 구현 부분 끝  ##############################
###################################################################
"""
[주의!]
하단 코드는 수정하지 마세요
"""


##SEED값 고정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)


if DEVICE.type == "cuda":
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
 


sentence = ("There are many variations of passages of Lorem Ipsum available"
            "but the majority have suffered alteration in some form, by injected humour"
            "randomised words which don't look even slightly believable."
            "If you are going to use a passage of Lorem Ipsum"
            "you need to be sure there isn't anything embarrassing hidden in the middle of text.")

# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10
learning_rate = 0.001

# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot).to(DEVICE)
Y = torch.LongTensor(y_data).to(DEVICE)

model = MyModel(dic_size, hidden_size).to(DEVICE)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)


# start training
for i in range(1, 601):
    optimizer.zero_grad()
    outputs = model(X).to(DEVICE)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
    if i % 100 == 0:
        print("Output Result: ", predict_str)
        print("Loss value: \n", loss.item())


Output Result:        o                  e  e  e     e  e     eo     o  e               e  e  e  e          e          e  e          e  e  e     e                e     e  e         o           e         e         e                         e    e  e     eo            e  e  e           o     o          e      e         eo e             e  e 
Loss value: 
 2.8206403255462646
Output Result:  aer   nl sooe baviagthn snf Lortagetiof Lorem Ipsumeavenlaliarie thedmavormth hobe suffere  aleomesiln tn toms efrmn wh bojeme   hndsureardanised tormg torbg donrt loof eves tnightli baliogeblebff Lor mne toisg torim da oassage of Lorem Ipsumeav s d  to in mfre toi e inndt hns ning tiie tasiigg hinga  in toe maleye of Loreh
Loss value: 
 1.7285345792770386
Output Result:  oere mne many varietions of passaget of Lorem Ipsumyavailablebut the majority have suffered altoration in some form, by injected humourrandomised words which don't look even tlightly believable.If you are going to use a passage of Lor